# Centrality Algorithms

<a target="_blank" href="https://colab.research.google.com/github/neo4j/graph-data-science-client/blob/main/examples/centrality-algorithms.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This Jupyter notebook is hosted [here](https://github.com/neo4j/graph-data-science-client/blob/main/examples/centrality-algorithms.ipynb) in the Neo4j Graph Data Science Client Github repository.

Centrality algorithms are used to understand the role or influence of particular nodes in a graph. The notebook shows the application of centrality algorithms using the `graphdatascience` library on the Airline travel reachability network dataset that can be downloaded [here](https://snap.stanford.edu/data/reachability.html).

This notebook will show how you can apply eigenvector centrality, betweenness centrality, degree centrality and closeness centrality on a graph dataset.

### Setup

We start by importing our dependencies and setting up our GDS client connection to the database.

In [ ]:
# Install necessary dependencies
# %pip install graphdatascience pandas

In [ ]:
from graphdatascience import GraphDataScience
import pandas as pd
import os

In [ ]:
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")
NEO4J_AUTH = None
if os.environ.get("NEO4J_USER") and os.environ.get("NEO4J_PASSWORD"):
    NEO4J_AUTH = (
        os.environ.get("NEO4J_USER"),
        os.environ.get("NEO4J_PASSWORD"),
    )

gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)

### Importing the dataset

We import the dataset as a pandas dataframe first. We deal with two files here. The file `reachability-meta.csv.gz` stores the names of the cities and their information while the file `reachability.txt.gz` stores the edges of the graph. An edge exists from city `i` to city `j` if the estimated airline travel time is less than a threshold.


In [ ]:
nodes_info_df = pd.read_csv("https://snap.stanford.edu/data/reachability-meta.csv.gz", compression="gzip")
nodes_info_df.head()

In [ ]:
routes_df = pd.read_csv(
    "https://snap.stanford.edu/data/reachability.txt.gz",
    sep=" ",
    skiprows=6,
    header=None,
    compression="gzip",
    names=["Origin", "Destination", "Weight"],
)
routes_df.head()

Next, we load this data (nodes and edges) into a Graph Database and a GDS graph.

In [ ]:
gds.run_cypher(
    "UNWIND $nodes AS node CREATE (n:City {node_id: node.node_id, name: node.name, population: node.metro_pop})",
    params={"nodes": nodes_info_df.to_dict("records")},
)

gds.run_cypher(
    """
    UNWIND $rels AS rel 
    MATCH (source:City {node_id: rel.Origin}), (target:City {node_id: rel.Destination}) 
    CREATE (source)-[:HAS_FLIGHT_TO]->(target)
    """,
    params={"rels": routes_df.to_dict("records")},
)

In [ ]:
G, result = gds.graph.project("airline", "City", "HAS_FLIGHT_TO")

print(f"The projection took {result['projectMillis']} ms")

# We can use convenience methods on `G` to check if the projection looks correct
print(f"Graph '{G.name()}' node count: {G.node_count()}")
print(f"Graph '{G.name()}' node labels: {G.node_labels()}")

### Eigenvector Centrality

[Eigenvector centrality](https://neo4j.com/docs/graph-data-science/current/algorithms/eigenvector-centrality/) measures the importance or influence of a node based on its connections to other nodes in the network. A higher eigenvector centrality score suggests that a node is more central and influential within the network.

For our dataset, eigenvector centrality can help identify airports that are not only well-connected themselves but also have connections to other important airports. Nodes with high eigenvector centrality are likely to be major hubs or airports with extensive connectivity.

In [ ]:
eigenvector_centrality_result = gds.eigenvector.mutate(G, maxIterations=100, mutateProperty="eigenvectorCentrality")

In [ ]:
# We can verify that the eigenvectorCentrality was mutated
G.node_properties()

We can see if our implementation converged or not and if converged, the number of iterations it took using the below code:

In [ ]:
if eigenvector_centrality_result.didConverge:
    print(
        f"The number of iterations taken by Eigenvector Centrality to run is {eigenvector_centrality_result.ranIterations}."
    )
else:
    print("Algorithm did not converge!")

We can also see the distribution of the eigenvector centrality measures using the below code. This will show us the minimum, maximum, mean and other statistical values for our centrality measure.

In [ ]:
eigenvector_centrality_result.centralityDistribution

In [ ]:
gds.graph.nodeProperties.write(G, ["eigenvectorCentrality"])

Using the results from eigenvector centrality, we can now look up the top 20 cities with airports that have major hubs or airports with extensive connectivity.

In [ ]:
def display_top_20_cities(centrality_measure):
    """
    Function to execute the Cypher query to retrieve the top 20 cities with the highest centrality measure.
    """
    query = f"""
    MATCH (n:City)
    RETURN n.node_id AS node_id, n.name AS name, n.population AS population, n.{centrality_measure} AS {centrality_measure}
    ORDER BY n.{centrality_measure} DESC
    LIMIT 20
    """
    result = gds.run_cypher(query)

    # Display the result
    print(result)


display_top_20_cities("eigenvectorCentrality")

### Betweenness Centrality

[Betweenness Centrality](https://neo4j.com/docs/graph-data-science/current/algorithms/betweenness-centrality/) quantifies the importance of a node as a bridge or intermediary in the network. It measures how often a node lies on the shortest path between other pairs of nodes. 

For our dataset, cities/airports with high betweenness centrality serve as crucial transfer points or connecting hubs between airports that might not have direct flights between them. They play a significant role in facilitating the flow of air travel and can be vital for overall network connectivity.

In [ ]:
betweenness_centrality_result = gds.betweenness.mutate(G, mutateProperty="betweennessCentrality")

In [ ]:
# We can verify that the betweennessCentrality was mutated
G.node_properties()

We can also see the distribution of the betweenness centrality measures using the below code. This will show us the minimum, maximum, mean and other statistical values for our centrality measure.

In [ ]:
betweenness_centrality_result.centralityDistribution

In [ ]:
gds.graph.nodeProperties.write(G, ["betweennessCentrality"])

Using the results from betweenness centrality, we can now look up the top 20 cities with airports that serve as crucial transfer points or connecting hubs between airports that might not have direct flights between them.

In [ ]:
display_top_20_cities("betweennessCentrality")

### Degree Centrality

[Degree Centrality](https://neo4j.com/docs/graph-data-science/current/algorithms/degree-centrality/) measures the number of connections (edges) a node has in the network. 

For our dataset, cities with high degree centrality have a large number of direct flight connections to other cities. They represent cities that have many direct destinations or are frequently used for direct travel. Degree centrality provides insights into the prominence and connectivity of individual airports within the network.

In [ ]:
degree_centrality_result = gds.degree.mutate(G, mutateProperty="degreeCentrality")

In [ ]:
# We can verify that the degreeCentrality was mutated
G.node_properties()

Similar to above, we can also see the distribution of the degree centrality measures using the below code. This will show us the minimum, maximum, mean and other statistical values for our centrality measure.

In [ ]:
degree_centrality_result.centralityDistribution

In [ ]:
gds.graph.nodeProperties.write(G, ["degreeCentrality"])

Finally, using the results from degree centrality, we can now look up the top 20 cities with airports that have a large number of direct flights.

In [ ]:
display_top_20_cities("degreeCentrality")

### Cleanup

Before finishing we can clean up the example data from both the GDS in-memory state and the database.

In [ ]:
# Cleanup GDS
G.drop()

In [ ]:
# Cleanup database
gds.run_cypher("MATCH (n:City) DETACH DELETE n")

### References
- For the network:
Brendan J. Frey and Delbert Dueck. "Clustering by passing messages between data points." Science 315.5814 (2007): 972-976.

- For the city metadata (metropolitan population, latitude, and longitude):
Austin R. Benson, David F. Gleich, and Jure Leskovec. "Higher-order Organization of Complex Networks." Science, 353.6295 (2016): 163–166.

- Notebook contributed by [Kedar Ghule](https://github.com/kedarghule)